# AI Agent Workflow Demo

This notebook demonstrates two distinct AI agent architectures deployed on our
Kubernetes AI Stack:

1. **CrewAI Research Pipeline** -- A multi-agent system where three specialised agents
   (Researcher, Writer, Reviewer) collaborate to produce a research report.
2. **LangGraph Stateful Workflow** -- A graph-based workflow with conditional routing
   that classifies queries and decides whether to use RAG retrieval or direct generation.

Both systems use OpenAI-compatible APIs backed by LocalAI, making them fully
self-hosted and model-agnostic.

> **Note:** The agent code lives in `agents/crewai-research-agent/` and
> `agents/langgraph-workflow/` respectively.

In [ ]:
"""
Import Dependencies
-------------------
We import CrewAI and LangGraph components. These are also used in the actual
agent services deployed to Kubernetes. The LLM backend is configured via
environment variables pointing to the LocalAI service.
"""

import os
import json
from pprint import pprint

# Set environment variables for the LLM backend (LocalAI in Kubernetes)
# In production these come from Kubernetes ConfigMaps/Secrets.
os.environ.setdefault("LLM_BASE_URL", "http://localai.ai-stack.svc.cluster.local:8080/v1")
os.environ.setdefault("LLM_API_KEY", "sk-no-key-required")
os.environ.setdefault("LLM_MODEL_NAME", "gpt-3.5-turbo")
os.environ.setdefault("LLM_TEMPERATURE", "0.7")
os.environ.setdefault("LLM_MAX_TOKENS", "4096")
os.environ.setdefault("QDRANT_URL", "http://qdrant.ai-stack.svc.cluster.local:6333")

# CrewAI imports
try:
    from crewai import Agent, Task, Crew, Process
    from langchain_openai import ChatOpenAI
    CREWAI_AVAILABLE = True
    print("CrewAI and LangChain loaded successfully")
except ImportError as e:
    CREWAI_AVAILABLE = False
    print(f"CrewAI not available: {e}")
    print("Install with: pip install crewai langchain-openai")

# LangGraph imports
try:
    from langgraph.graph import END, START, StateGraph
    LANGGRAPH_AVAILABLE = True
    print("LangGraph loaded successfully")
except ImportError as e:
    LANGGRAPH_AVAILABLE = False
    print(f"LangGraph not available: {e}")
    print("Install with: pip install langgraph")

## CrewAI Research Pipeline

The CrewAI pipeline uses three specialised agents that collaborate sequentially:

| Agent | Role | Responsibility |
|-------|------|----------------|
| **Researcher** | Senior Research Analyst | Gather and analyse information on the topic |
| **Writer** | Technical Writer | Draft a structured report from the research |
| **Reviewer** | Quality Reviewer | Review the report for accuracy and completeness |

The pipeline is configured via `config.yaml` and uses `langchain_openai.ChatOpenAI`
as the LLM backend, pointed at our LocalAI instance.

```
Research Task --> Writing Task (context: research) --> Review Task (context: research + writing)
```

In [ ]:
# ---------------------------------------------------------------------------
# CrewAI Agent Creation and Crew Execution
# ---------------------------------------------------------------------------
# This mirrors the code in agents/crewai-research-agent/agents.py and tasks.py

# Research topic for this demo
RESEARCH_TOPIC = "Best practices for deploying LLMs on Kubernetes with limited GPU resources"

if CREWAI_AVAILABLE:
    # Step 1: Configure the shared LLM (pointed at LocalAI)
    llm = ChatOpenAI(
        model=os.getenv("LLM_MODEL_NAME", "gpt-3.5-turbo"),
        base_url=os.getenv("LLM_BASE_URL", "http://localai:8080/v1"),
        api_key=os.getenv("LLM_API_KEY", "sk-no-key-required"),
        temperature=0.7,
        max_tokens=4096,
        request_timeout=120,
    )

    # Step 2: Create the three specialised agents
    researcher = Agent(
        role="Senior Research Analyst",
        goal="Conduct thorough research and provide comprehensive analysis",
        backstory=(
            "You are an experienced research analyst specialising in AI/ML "
            "infrastructure and Kubernetes deployments. You excel at finding "
            "and synthesising technical information."
        ),
        allow_delegation=False,
        verbose=True,
        llm=llm,
    )

    writer = Agent(
        role="Technical Writer",
        goal="Transform research findings into clear, well-structured reports",
        backstory=(
            "You are a skilled technical writer with deep knowledge of cloud-native "
            "technologies. You create documentation that is both technically accurate "
            "and accessible to a broad audience."
        ),
        allow_delegation=False,
        verbose=True,
        llm=llm,
    )

    reviewer = Agent(
        role="Quality Reviewer",
        goal="Ensure reports are accurate, complete, and well-organised",
        backstory=(
            "You are a meticulous quality reviewer who checks technical content "
            "for accuracy, completeness, and clarity. You provide constructive "
            "feedback and identify gaps in reasoning."
        ),
        allow_delegation=False,
        verbose=True,
        llm=llm,
    )

    # Step 3: Define tasks with context chaining
    research_task = Task(
        description=(
            f"Research the topic: '{RESEARCH_TOPIC}'. Gather key findings, "
            "best practices, and real-world examples. Focus on quantisation "
            "techniques (GGML, GPTQ, AWQ), memory optimisation, and orchestration "
            "patterns for self-hosted LLM inference."
        ),
        expected_output=(
            "A structured research brief with key findings, data points, "
            "and references organised by subtopic."
        ),
        agent=researcher,
    )

    writing_task = Task(
        description=(
            f"Write a technical report on '{RESEARCH_TOPIC}' based on the "
            "research findings. Include an executive summary, detailed sections, "
            "and actionable recommendations."
        ),
        expected_output=(
            "A well-structured technical report in markdown format with "
            "headings, bullet points, and a clear conclusion."
        ),
        agent=writer,
        context=[research_task],  # Writer receives researcher's output
    )

    review_task = Task(
        description=(
            "Review the technical report for accuracy, completeness, and clarity. "
            "Check that all claims are supported and the recommendations are practical. "
            "Provide a final quality assessment."
        ),
        expected_output=(
            "A reviewed report with quality score, identified issues, and "
            "the final approved version of the report."
        ),
        agent=reviewer,
        context=[research_task, writing_task],  # Reviewer sees both
    )

    # Step 4: Assemble the Crew with sequential execution
    crew = Crew(
        agents=[researcher, writer, reviewer],
        tasks=[research_task, writing_task, review_task],
        process=Process.sequential,  # Tasks run in order
        verbose=True,
    )

    print("CrewAI pipeline assembled successfully!")
    print(f"  Topic:    {RESEARCH_TOPIC}")
    print(f"  Agents:   {len(crew.agents)}")
    print(f"  Tasks:    {len(crew.tasks)}")
    print(f"  Process:  Sequential")
    print(f"  LLM:      {os.getenv('LLM_MODEL_NAME')}")
    print(f"\nTo execute: result = crew.kickoff()")
    print("(Execution requires a running LocalAI instance)")

    # Uncomment to actually run the crew (requires LocalAI to be reachable):
    # result = crew.kickoff()
    # print(result)

else:
    print("CrewAI is not installed. Showing the pipeline structure:")
    print()
    print("Pipeline: Researcher --> Writer --> Reviewer")
    print(f"Topic:    {RESEARCH_TOPIC}")
    print()
    print("Agents:")
    print("  1. Senior Research Analyst -- gathers information on the topic")
    print("  2. Technical Writer -- drafts a structured report from research")
    print("  3. Quality Reviewer -- validates accuracy and completeness")
    print()
    print("Install CrewAI to run: pip install crewai langchain-openai")

## LangGraph Stateful Workflow

The LangGraph workflow implements an intelligent query router with four nodes:

```
START --> classify_query --> [conditional routing]
                              |
                              +-- route="rag" -----> retrieve_context --+
                              |                                         |
                              +-- route="direct" --------------------+  |
                                                                     |  |
                                                                     v  v
                                                               generate_response
                                                                     |
                                                                     v
                                                               check_quality
                                                                     |
                                                                     v
                                                                    END
```

**Key concepts:**
- **State**: A `TypedDict` (`WorkflowState`) that flows through the graph with fields for
  query, classification, context, response, route, and quality metrics.
- **Nodes**: Pure functions that receive state and return partial state updates.
- **Conditional edges**: The routing decision determines whether to fetch context from
  Qdrant (RAG path) or answer directly.

In [ ]:
# ---------------------------------------------------------------------------
# LangGraph Workflow Construction and Execution
# ---------------------------------------------------------------------------
# This mirrors the code in agents/langgraph-workflow/graph.py, state.py, and nodes.py

from typing import TypedDict, Any, Annotated

if LANGGRAPH_AVAILABLE:

    # Step 1: Define the state schema (mirrors state.py)
    class WorkflowState(TypedDict, total=False):
        """Typed state flowing through the LangGraph workflow."""
        query: str              # Original user question
        classification: str     # "factual", "analytical", or "creative"
        context: list[str]      # Retrieved context chunks (RAG path only)
        response: str           # Generated answer
        route: str              # "rag" or "direct"
        quality_score: int      # 0-100 quality score
        quality_passed: bool    # Whether the response passed quality gate

    # Step 2: Define node functions (simplified versions of nodes.py)
    def classify_query(state: WorkflowState) -> dict[str, Any]:
        """Classify the query type and determine routing."""
        query = state["query"]
        # In production, this calls the LLM for classification.
        # Here we use a simplified heuristic for demonstration.
        factual_keywords = ["what", "define", "explain", "how does", "describe"]
        is_factual = any(kw in query.lower() for kw in factual_keywords)

        classification = "factual" if is_factual else "analytical"
        # Route to RAG for factual queries that benefit from context
        route = "rag" if is_factual else "direct"

        print(f"  [classify_query] classification={classification}, route={route}")
        return {"classification": classification, "route": route}

    def retrieve_context(state: WorkflowState) -> dict[str, Any]:
        """Retrieve supporting context from the vector store."""
        query = state["query"]
        # In production, this queries Qdrant via vector similarity search.
        # Here we return simulated context for demonstration.
        context = [
            f"Context 1: Relevant information about '{query[:50]}' "
            "from the knowledge base. In production this comes from Qdrant.",
            "Context 2: Supporting data and examples that provide "
            "grounding for the LLM response.",
            "Context 3: Additional reference material related to the topic.",
        ]
        print(f"  [retrieve_context] Retrieved {len(context)} chunks")
        return {"context": context}

    def generate_response(state: WorkflowState) -> dict[str, Any]:
        """Generate a response, optionally using retrieved context."""
        query = state["query"]
        context = state.get("context", [])
        classification = state.get("classification", "factual")

        # In production, this calls the LLM via ChatOpenAI.
        if context:
            response = (
                f"Based on {len(context)} retrieved sources: "
                f"This is a {classification} query about '{query[:60]}'. "
                "The answer would be generated by the LLM using the "
                "retrieved context for grounding."
            )
        else:
            response = (
                f"Direct answer for this {classification} query about "
                f"'{query[:60]}'. No external context was needed."
            )
        print(f"  [generate_response] Generated {len(response)} chars "
              f"(context_chunks={len(context)})")
        return {"response": response}

    def check_quality(state: WorkflowState) -> dict[str, Any]:
        """Validate the quality of the generated response."""
        response = state.get("response", "")
        # In production, this calls the LLM to score the response.
        score = min(85, len(response))  # Simplified scoring
        passed = score >= 60
        print(f"  [check_quality] score={score}, passed={passed}")
        return {"quality_score": score, "quality_passed": passed}

    # Step 3: Build the graph (mirrors graph.py)
    def route_decision(state: WorkflowState) -> str:
        """Conditional edge: choose next node based on routing decision."""
        route = state.get("route", "direct")
        if route == "rag":
            return "retrieve_context"
        return "generate_response"

    graph = StateGraph(WorkflowState)

    # Register nodes
    graph.add_node("classify_query", classify_query)
    graph.add_node("retrieve_context", retrieve_context)
    graph.add_node("generate_response", generate_response)
    graph.add_node("check_quality", check_quality)

    # Define edges
    graph.add_edge(START, "classify_query")
    graph.add_conditional_edges(
        "classify_query",
        route_decision,
        {
            "retrieve_context": "retrieve_context",
            "generate_response": "generate_response",
        },
    )
    graph.add_edge("retrieve_context", "generate_response")
    graph.add_edge("generate_response", "check_quality")
    graph.add_edge("check_quality", END)

    # Compile the graph
    workflow = graph.compile()
    print("LangGraph workflow compiled successfully!")

    # Step 4: Execute the workflow with sample queries
    test_queries = [
        "What is a Kubernetes Service and how does it work?",
        "Compare the performance of Mistral 7B and LLaMA 3 for code generation tasks.",
    ]

    for query in test_queries:
        print(f"\n{'='*65}")
        print(f"Query: {query}")
        print('='*65)
        result = workflow.invoke({"query": query})
        print(f"\nResult:")
        print(f"  Classification:  {result.get('classification')}")
        print(f"  Route:           {result.get('route')}")
        print(f"  Context chunks:  {len(result.get('context', []))}")
        print(f"  Quality score:   {result.get('quality_score')}")
        print(f"  Quality passed:  {result.get('quality_passed')}")
        print(f"  Response:        {result.get('response', '')[:120]}...")

else:
    print("LangGraph is not installed. Showing the workflow structure:")
    print()
    print("Nodes: classify_query -> [route] -> retrieve_context / generate_response -> check_quality")
    print()
    print("Install LangGraph to run: pip install langgraph")

## Results

The two agent architectures serve different purposes:

| Feature | CrewAI | LangGraph |
|---------|--------|-----------|
| **Architecture** | Multi-agent collaboration | Graph-based state machine |
| **Execution** | Sequential task pipeline | Conditional node traversal |
| **State** | Passed via task context | Explicit `TypedDict` schema |
| **Routing** | Fixed pipeline order | Dynamic conditional edges |
| **Best for** | Complex multi-step research | Query routing and classification |
| **LLM calls** | 3+ (one per agent per task) | 2-3 (classify + generate + quality) |

## Conclusions

### CrewAI Research Pipeline
- The multi-agent approach excels at complex, multi-step tasks where different
  expertise is needed at each stage.
- Sequential processing ensures each agent builds on the previous agent's output.
- The `config.yaml`-driven design allows tuning agent behaviour without code changes.
- Trade-off: higher latency (3+ LLM calls) but richer, more polished output.

### LangGraph Stateful Workflow
- The graph-based approach is ideal for request routing and conditional logic.
- Conditional edges enable dynamic branching (RAG vs. direct answer).
- The explicit state schema (`WorkflowState`) makes data flow transparent and testable.
- The quality gate node provides a safety mechanism to catch low-quality responses.
- Trade-off: lower latency but less depth per response.

### Deployment on Kubernetes
Both agent systems are containerised and deployed as Kubernetes pods, sharing the
same LocalAI backend. This architecture allows:
- Independent scaling of agent services vs. LLM inference.
- Graceful fallback when Qdrant or LocalAI is temporarily unavailable.
- Centralised configuration via Kubernetes ConfigMaps and Secrets.